***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [36]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-998c  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [37]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes


In [38]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 12:35 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [40]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [41]:
spark

In [42]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'chamber_of_secrets' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [43]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [47]:
# adding our python module to the cluster
sc.clearFiles()
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
sys.path.remove('/home/sergey')

AttributeError: 'SparkContext' object has no attribute 'clearFiles'

In [45]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token, offset):
    return int(_hash(token),16) % NUM_BUCKETS + offset

# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs 
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # YOUR CODE HERE
    #stopwords removal
    diff = set(tokens)-all_stopwords
    tokens = [item for item in tokens if item in diff]
    #counter 
    counter = Counter()
    for token in tokens:
        counter[token] += 1
    result = []
    for word in counter.keys():
        result.append( (word, (id, counter[word])) )
    return result

#caculate document frequency
def calc_dl(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    # YOUR CODE HERE
    #stopwords removal
    diff = set(tokens)-all_stopwords
    tokens = [item for item in tokens if item in diff]
    return (id, len(tokens))

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples 
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    # YOUR CODE HERE
    sorted_pl = sorted(unsorted_pl, key = lambda x: x[0])
    return sorted_pl 

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    # YOUR CODE HERE
    result = postings.mapValues(lambda x: len(x))
    return result

# calc doc_nf
def get_tfidf_norm(tup, w2df_dict, dl):
    term = tup[0]
    pls = tup[1]
    updates = []
    for item in pls: # (doc_id, freq)
        tf = item[1] / dl[item[0]]
        idf = np.log10( len(dl) / w2df_dict.get(term,len(dl)))
        updates.append((item[0], (tf * idf)**2))
    return updates

def partition_postings_and_write(postings, offset,bucket_name):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and 
        offsets its posting list was written to. See `write_a_posting_list` for 
        more details.
    '''
    # YOUR CODE HERE
    token2bucket = postings.map(lambda x: (token2bucket_id(x[0], offset), x))
    buckets = token2bucket.groupByKey().mapValues(list)
    result = buckets.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))
    return result

#PAGERANK
def generate_graph(pages):
    ''' Compute the directed graph generated by wiki links.
    Parameters:
    -----------
      pages: RDD
        An RDD where each row consists of one wikipedia articles with 'id' and 
        'anchor_text'.
    Returns:
    --------
      edges: RDD
        An RDD where each row represents an edge in the directed graph created by
        the wikipedia links. The first entry should the source page id and the 
        second entry is the destination page id. No duplicates should be present. 
      vertices: RDD
        An RDD where each row represents a vetrix (node) in the directed graph 
        created by the wikipedia links. No duplicates should be present. 
    '''
    # YOUR CODE HERE
    vertices = pages.flatMap(lambda x: [x.id] + [row.id for row in x.anchor_text]).distinct()
    edges = pages.flatMap(lambda x: [(x.id, row.id) for row in x.anchor_text]).distinct()
    vertices = vertices.map(lambda x: Row(id = x))
    edges = edges.map(lambda x: Row(src = x[0], dst = x[1]))
    return edges, vertices

In [14]:
offset = 124

In [15]:
#TEXT
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

# #Create term_totals
# without_doc_id_text = word_counts_text.map(lambda x: (x[0], x[1][1])) 
# grouped_list_text = without_doc_id_text.reduceByKey(lambda x,y: x+y)
# term_total_text = Counter(dict(grouped_list_text.collect()))

# #Create dl
# dl_text = dict(doc_text_pairs.map(lambda x: calc_dl(x.text, x.id)).collect())

# #
# postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# postings_text = postings_text.filter(lambda x: len(x[1] > 50))

# # global statistics
# w2df_text = calculate_df(postings_text)
# w2df_text_dict = w2df_text.collectAsMap()

# # normalize factor for cosine similarity on tfidf
# nf_docs_text = postings_text.flatMap(lambda x: get_tfidf_norm(x, w2df_text_dict, text)).reduceByKey(lambda x, y: x+y)
# nf_docs_text = nf_docs_text.collectAsMap()

# #
# _ = partition_postings_and_write(postings_text, offset*0,bucket_name).collect()

# # collect all posting lists locations into one super-set
# super_posting_locs = defaultdict(list)
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
#     if not blob.name.endswith("pickle"):
#         continue
#     with blob.open("rb") as f:
#         posting_locs = pickle.load(f)
#         for k, v in posting_locs.items():
#             super_posting_locs[k].extend(v)

# #creating and writing the index
# # Create inverted index instance
# inverted_text = InvertedIndex()
# # Adding the posting locations dictionary to the inverted index
# inverted_text.posting_locs = super_posting_locs
# # Add the token - df dictionary to the inverted index
# inverted_text.df = w2df_text_dict
# # Add term total
# inverted_text.term_total = term_total_text
# # Add dl
# inverted_text.dl = dl_text
# # Add title
# inverted_text.title = title_dict
# #add nf_docs for cosine
# inverted_text.nf_docs = nf_docs_text
# # write the global stats out
# inverted_text.write_index('.', 'text_index')

# # upload to gs
# index_src = "text_index.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
# !gsutil cp $index_src $index_dst
# !gsutil ls -lh $index_dst

In [16]:
#Create term_totals
# without_doc_id_text = word_counts_text.map(lambda x: (x[0], x[1][1])) 
# grouped_list_text = without_doc_id_text.reduceByKey(lambda x,y: x+y)
# term_total_text = Counter(dict(grouped_list_text.collect()))

In [18]:
#Create dl
dl_text = doc_text_pairs.map(lambda x: calc_dl(x.text, x.id)).collectAsMap()

Exception in thread "serve RDD 30" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [19]:
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_text = postings_text.filter(lambda x: len(x[1]) > 50)

In [ ]:
w2df_text = calculate_df(postings_text)
w2df_text_dict = w2df_text.collectAsMap()

In [ ]:
nf_docs_text = postings_text.flatMap(lambda x: get_tfidf_norm(x, w2df_text_dict, dl_text)).reduceByKey(lambda x, y: x+y)
nf_docs_text = nf_docs_text.collectAsMap()

In [35]:
_ = partition_postings_and_write(postings_text, offset*0, bucket_name).collect()

# # collect all posting lists locations into one super-set
# super_posting_locs = defaultdict(list)
# for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
#     if not blob.name.endswith("pickle"):
#         continue
#     with blob.open("rb") as f:
#         posting_locs = pickle.load(f)
#         for k, v in posting_locs.items():
#             super_posting_locs[k].extend(v)

22/01/09 17:08:48 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 30.0 (TID 2309) (cluster-998c-w-0.c.imperial-arcade-329710.internal executor 30): java.lang.RuntimeException: Stream '/files/inverted_index_gcp_1.py' was not found.
	at org.apache.spark.network.client.TransportResponseHandler.handle(TransportResponseHandler.java:243)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:142)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:53)
	at io.netty.channel.SimpleChannelInboundHandler.channelRead(SimpleChannelInboundHandler.java:99)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)
	at io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)
	at io.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 30.0 failed 4 times, most recent failure: Lost task 0.3 in stage 30.0 (TID 2318) (cluster-998c-w-0.c.imperial-arcade-329710.internal executor 30): java.lang.RuntimeException: Stream '/files/inverted_index_gcp_1.py' was not found.
	at org.apache.spark.network.client.TransportResponseHandler.handle(TransportResponseHandler.java:243)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:142)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:53)
	at io.netty.channel.SimpleChannelInboundHandler.channelRead(SimpleChannelInboundHandler.java:99)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)
	at io.netty.channel.AbstractChannelHandlerContext.invokeC

In [23]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

In [24]:
#creating and writing the index
# Create inverted index instance
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_text_dict
# Add dl
inverted_text.dl = dl_text
#add nf_docs for cosine
inverted_text.nf_docs = nf_docs_text
# write the global stats out
inverted_text.write_index('.', 'text_index')

In [25]:
# upload to gs
index_src = "text_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

Copying file://text_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][195.8 MiB/195.8 MiB]                                                
Operation completed over 1 objects/195.8 MiB.                                    
195.76 MiB  2022-01-09T16:40:47Z  gs://chamber_of_secrets/postings_gcp/text_in

In [26]:
#TITLE
word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))

#Create term_totals
without_doc_id_title = word_counts_title.map(lambda x: (x[0], x[1][1])) 
grouped_list_title = without_doc_id_title.reduceByKey(lambda x,y: x+y)
term_total_title = Counter(dict(grouped_list_title.collect()))

#Create dl
dl_title = dict(doc_title_pairs.map(lambda x: calc_dl(x.title, x.id)).collect())

#
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

# global statistics
w2df_title = calculate_df(postings_title)
w2df_title_dict = w2df_title.collectAsMap()

# normalize factor for cosine similarity on tfidf
nf_docs_title = postings_title.flatMap(lambda x: get_tfidf_norm(x, w2df_title_dict, dl_title)).reduceByKey(lambda x, y: x+y)
nf_docs_title = nf_docs_title.collectAsMap()

#
_ = partition_postings_and_write(postings_title, offset*1,bucket_name).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

#creating and writing the index
inverted_title = InvertedIndex()
inverted_title.posting_locs = super_posting_locs
inverted_title.df = w2df_title_dict
inverted_title.dl = dl_title
inverted_title.nf_docs = nf_docs_title
inverted_title.write_index('.', 'title_index')

# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][210.4 MiB/210.4 MiB]                                                
Operation completed over 1 objects/210.4 MiB.                                    
210.41 MiB  2022-01-09T16:52:04Z  gs://chamber_of_secrets/postings_gcp/title_

In [ ]:
#ANCHOR
doc_anchor_pairs = doc_anchor_pairs.map(lambda x: Row(anchor_text = ' '.join([row.text for row in x.anchor_text]), id = x.id))
word_counts_anchor = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))

#Create term_totals
without_doc_id_anchor = word_counts_anchor.map(lambda x: (x[0], x[1][1])) 
grouped_list_anchor = without_doc_id_anchor.reduceByKey(lambda x,y: x+y)
term_total_anchor = Counter(dict(grouped_list_anchor.collect()))

#Create dl
dl_anchor = dict(doc_anchor_pairs.map(lambda x: calc_dl(x.anchor_text, x.id)).collect())

#
postings_anchor = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

# global statistics
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()

# normalize factor for cosine similarity on tfidf
nf_docs_anchor = postings_anchor.flatMap(lambda x: get_tfidf_norm(x, w2df_anchor_dict, dl_anchor)).reduceByKey(lambda x, y: x+y)
nf_docs_anchor = nf_docs_anchor.collectAsMap()

#
_ = partition_postings_and_write(postings_anchor, offset*2,bucket_name).collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

#creating and writing the index
inverted_anchor = InvertedIndex()
inverted_anchor.posting_locs = super_posting_locs
inverted_anchor.df = w2df_anchor_dict
inverted_anchor.dl = dl_anchor
inverted_anchor.nf_docs = nf_docs_anchor
inverted_anchor.write_index('.', 'anchor_index')

# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.93 GiB     gs://sise_ir_assignment3/postings_gcp
